In [63]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

def get_num_folds(cv_percent=20):
    import math
    return math.floor(X.shape[0] / (X.shape[0] * (cv_percent / 100)))

### Data preprocessing

This dataset contains few non-numerical features. Here we restructure cathegorical data into boolean-valued columns to be able to include them into model in the numerical form

In [65]:
df = pd.read_csv('forestfires.csv')
df.dropna()

df = pd.get_dummies(df)

### Preparing the subsets

In [66]:
X = df.drop(["X", "Y", "area"], axis=1).values
y = df["area"].values

scaler = RobustScaler()
X = scaler.fit_transform(X)

For later interpretation of the model performance, we split dataset into three subsets: <br>
   1 - train <br>
   2 - cross-validation [20%] <br>
   3 - test [20%] <br>

### Training regression model

Train a Linear Regression model and perform regression analysis using the coefficient of determination ('r2').  

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

cv_results = np.mean(cross_val_score(lr, X_test, y_test, cv=get_num_folds(),  scoring='r2'))  # To see possible scorings: sorted(sklearn.metrics.SCORERS.keys()).
print(cv_results)

-2.4275849438861123
